# Sembla Agent Demo

In [70]:
import os

from sembla.schemas.system import SystemState, Message, UserQuery, ResponseSchema, ActionCall, MemoryState, AgentResponse, Action
from sembla.system import init_sequential_agent_system
from sembla.conversation_history import update_conversation_history
from sembla.llm.openai.chat_completion import generate_chat_completion
from sembla.prompt.utils import create_system_prompt


%load_ext autoreload
%autoreload 2

# Set the OpenAI API key here or in your environment
OPENAI_API_KEY = None

if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**Note:** User query should probably be a dict so that we can support things like multimodal systems.

In [71]:
from sembla.actions.functions import submit_completed_task, list_directory


def functions_to_actions(functions):
    actions = []
    for function in functions:
        actions.append(Action.from_callable(function))
    return actions

actions = functions_to_actions([submit_completed_task, list_directory])

response_example = ResponseSchema(
    goal="<Here should be a short, high-level description of what you're trying to achieve with this set of actions>",
    objectives=[
        "<This is a list where each element is a more specific task or action that helps to achieve the overall goal>",
    ],
    observations=[
        "<This is a list where each element represents an important piece of information or context that was observed or discovered during the process>",
    ],
    action=ActionCall(
        name="<This is the next action that will be taken towards achieving the objectives>",
        parameters={
            "arg1": "<This is an example of a parameter that the action takes>",
        }
    )
)


instructions = """\
You are an autonomous agent that performs tasks for users.
"""
system_prompt = create_system_prompt(
    instructions=instructions,
    actions=actions,
    response_example=response_example
)

user_query = "How many files are in the current directory?"
user_query = UserQuery(raw_query=user_query)
initial_state = SystemState(
    user_query=user_query,
    system_prompt=system_prompt,
    response_schema=ResponseSchema,
    memory=MemoryState(
        conversation_history=[
            Message(role="system", content=system_prompt),
        ]
    ),
    actions=actions,
)

print(initial_state.to_json())



{
    "task": {
        "name": null,
        "description": null,
        "status": "UNDEFINED",
        "max_cycles": null,
        "current_cycle": 0
    },
    "model": {
        "name": "gpt-3.5-turbo",
        "temperature": 0.2,
        "n": 1,
        "max_tokens": 2000,
        "frequency_penalty": 0,
        "presence_penalty": 0
    },
    "system_prompt": "You are an autonomous agent that performs tasks for users.\n\nYou have the following actions available:\nsubmit_completed_task(*args, **kwargs): Submit completed task to the user.\nlist_directory(directory): List contents of `directory`.\n\nYour response must be in the following format:\n{\n  \"goal\": \"<Here should be a short, high-level description of what you're trying to achieve with this set of actions>\",\n  \"objectives\": [\n    \"<This is a list where each element is a more specific task or action that helps to achieve the overall goal>\"\n  ],\n  \"observations\": [\n    \"<This is a list where each element rep

In [72]:
def process_user_query(system_state: SystemState) -> SystemState:
    """Process the user query into a message and add it to the conversation buffer."""
    assert system_state.user_query is not None, "User query must be set."
    prompt = system_state.user_query.raw_query
    message = Message(role="user", content=prompt)
    new_memory = system_state.memory.copy(update={"conversation_buffer": [message]})
    new_state = system_state.copy(update={
        "memory": new_memory,
        "processed_prompt": prompt}
    )
    return new_state


In [73]:
from sembla.response.processor import parse_json_response
from sembla.actions.utils import execute_action_call


def process_agent_response(system_state: SystemState) -> SystemState:
    """Process the agent response into a message and add it to the conversation buffer."""
    assert system_state.agent_response is not None, "Agent response must be set."
    raw_response = system_state.agent_response.raw_response
    parsed_response = parse_json_response(response=raw_response, schema=system_state.response_schema)
    agent_response = AgentResponse(
        raw_response=raw_response,
        parsed_response=parsed_response,
    )
    message = Message(role="assistant", content=raw_response)
    new_memory = system_state.memory.copy(update={"conversation_buffer": [message]})
    new_state = system_state.copy(update={
        "memory": new_memory,
        "agent_response": agent_response,
    }
    )
    return new_state

def process_actions(system_state: SystemState) -> SystemState:
    """Process any actions that are included in the agent response."""
    assert system_state.agent_response is not None, "Agent response must be set."
    assert system_state.agent_response.parsed_response is not None, "Parsed response must be set."
    assert system_state.agent_response.parsed_response.action is not None, "Action must be set."
    action = system_state.agent_response.parsed_response.action
    if action:
        action_output = execute_action_call(available_actions=system_state.actions, action_call=action)
        new_state = system_state.copy(update={
            "action_output": action_output,
        }
        )
        action_message = Message(role="user", content=action_output.output)
        new_memory = new_state.memory.copy(update={"conversation_buffer": [action_message]})
        new_state = new_state.copy(update={
            "memory": new_memory,
        }
        )
        return new_state
    return system_state


In [74]:
agent_system = init_sequential_agent_system([
    process_user_query,
    update_conversation_history,
    generate_chat_completion,
    process_agent_response,
    update_conversation_history,
    process_actions,
    update_conversation_history,
])

In [83]:
final_state = agent_system(initial_state)
total_length = 80
for msg in final_state.memory.conversation_history:
    print("{:-^{}}".format(f" {msg.role.upper()} ", total_length))
    print()
    print(msg.content)
    print()

------------------------------------ SYSTEM ------------------------------------

You are an autonomous agent that performs tasks for users.

You have the following actions available:
submit_completed_task(*args, **kwargs): Submit completed task to the user.
list_directory(directory): List contents of `directory`.

Your response must be in the following format:
{
  "goal": "<Here should be a short, high-level description of what you're trying to achieve with this set of actions>",
  "objectives": [
    "<This is a list where each element is a more specific task or action that helps to achieve the overall goal>"
  ],
  "observations": [
    "<This is a list where each element represents an important piece of information or context that was observed or discovered during the process>"
  ],
  "action": {
    "name": "<This is the next action that will be taken towards achieving the objectives>",
    "parameters": {
      "arg1": "<This is an example of a parameter that the action takes>"
 